# Kết nối với Drive cá nhân


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Vì việc kết nối này sử dụng Drive cá nhân, nên t chưa tìm được cách chia sẻ dữ liệu để sử dụng chung. Đoạn code dưới đây sẽ tạo một bản sao của thư mục vào Drive cá nhân của mọi người và tải dữ liệu vào đó.
<br/>
Việc code thì có thể làm chung trong file này cũng được

In [ ]:
cd "/content/drive/MyDrive/"

/content/drive/MyDrive


In [ ]:
import os
from pathlib import Path


# Create directory if not exist
if not Path('./BTL_Data_Science').is_dir():
    os.mkdir('BTL_Data_Science')

In [ ]:
cd "./BTL_Data_Science/"

/content/drive/MyDrive/BTL_Data_Science


In [ ]:
!pwd

/content/drive/MyDrive/BTL_Data_Science


# Crawler

### Chuẩn bị

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
import os
import pathlib
import sys

import pandas as pd
import requests
import tqdm
from bs4 import BeautifulSoup


### Định nghĩa các hàm

In [ ]:
def get_links(topic_link=None, title_tag='h2'):
    """Get news URLs from topic link"""
    try:
        response = requests.get(topic_link)
        soup = BeautifulSoup(response.content, "html.parser")
        titles = soup.find_all(title_tag, class_='title-news')
        links = [link.find('a').attrs["href"] for link in titles]

        return links
    except Exception as e:
        print('Exception {} has happened when get links from URL: {}'.format(type(e).__name__, topic_link))


In [ ]:
def find_topic(soup=None):
    # A part of HTML which store topic of news
    t = soup.find('ul', class_='breadcrumb')  

    # Topic of news (VD: KhoaHoc)
    topic = (t.find('a').attrs["data-medium"])[5:]
    
    return topic


In [ ]:
def find_date(soup=None):
    t = soup.find(class_='header-content')  
    
    date = t.find(class_="date").get_text().split(", ")[1]

    return date
    

In [ ]:
def find_header(soup=None):
    header = soup.find(class_='title-detail').get_text()
    return header
    

In [ ]:
def find_tag(link):
   response = requests.get(link)
   soup = BeautifulSoup(response.content, "html.parser")
   header = soup.find(class_='title-detail').get_text()
   return header
   

In [ ]:
print(find_tag('https://vnexpress.net/tam-quan-trong-cua-tiem-vaccine-cho-phu-nu-truoc-trong-khi-mang-thai-4543330.html'))

Tầm quan trọng của tiêm vaccine cho phụ nữ trước, trong khi mang thai 


In [ ]:
def crawl_data(link=None, data_dir=Path('./data')):
    """Get data from URL and save to file"""
    try:
        if not Path(data_dir).is_dir():
            os.mkdir(data_dir)

        # Get HTML source code from link
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")

        topic = find_topic(soup)

        # Create new directory if this topic is not crawled
        topic_dir = data_dir / topic
        if not topic_dir.is_dir():
            os.mkdir(topic_dir)

        # Index file which stores URL of all crawled news in the same topic
        index_filename = '_Index.csv'.format(topic)  
        index_filepath = topic_dir / index_filename

        date = find_date(soup)
        #header = find_header(soup)

        # Text file which stores content of the news in link
        if not index_filepath.is_file():
            # If Index file is not exist
            index = 1
        else:
            df = pd.read_csv(index_filepath, header=None, index_col=None)
            index = int(df.iloc[-1, 0]) + 1

            # Stop if this link is crawled
            for crawled_link in df.iloc[:, 2]:
                if link == crawled_link:
                  return None

        text_filename = 'news{}.txt'.format(index)
        text_path = topic_dir / text_filename

        # Add new URL to Index file of the topic
        with open(index_filepath, mode='a+') as file:
            writer = csv.writer(file)
            writer.writerow([index, text_filename, str(link), date])

        # Write content to file
        with open(text_path, 'w', encoding='UTF-8') as f:
          # Write title of news
          f.write(soup.find('h1', class_='title-detail').text + '\n\n')

          # Write description of news
          f.write(soup.find('p', class_='description').text)

          # Write all content of news
          contents = soup.findAll('p', class_='Normal')
          for i in range(0, len(contents) - 1):
              f.write('\n' + contents[i].text)

    except Exception as e:
        print('Exception {} has happened when crawl data from URL: {}'.format(type(e).__name__, link))


In [ ]:
# URL = 'https://vnexpress.net/tam-quan-trong-cua-tiem-vaccine-cho-phu-nu-truoc-trong-khi-mang-thai-4543330.html'
# crawl_data(URL, DATA_DIR)


### Tiến hành crawl dữ liệu

In [ ]:
# a = 0  # News ID
# tl = []  # Crawled titles

CURRENT_DIR = Path(os.getcwd())  # Main directory, depends on where this code run (Google Colab or Local)
DATA_DIR = CURRENT_DIR / 'data3'
MIN_PAGE = 1
MAX_PAGE = 40


In [ ]:
if not DATA_DIR.is_dir():
  os.mkdir(DATA_DIR)

links2 = [
    'https://vnexpress.net/suc-khoe',
    'https://vnexpress.net/du-lich',
    'https://vnexpress.net/so-hoa',
    'https://vnexpress.net/kinh-doanh',
    'https://vnexpress.net/giai-tri',
    'https://vnexpress.net/the-thao'
]

links3 = [
    'https://vnexpress.net/thoi-su',
    'https://vnexpress.net/goc-nhin',
    'https://vnexpress.net/the-gioi',
    'https://vnexpress.net/khoa-hoc',
    'https://vnexpress.net/phap-luat',
    'https://vnexpress.net/giao-duc',
    'https://vnexpress.net/oto-xe-may',
    'https://vnexpress.net/hai',
]

try:
    
    for link in links2:
        news_links = []

        for i in tqdm.tqdm(list(range(MIN_PAGE, MAX_PAGE + 1)), desc=link):
            sub_link = link + '-p' + str(i)  # Topic link with page number
            news_links += get_links(sub_link, 'h2')

        for link in tqdm.tqdm(news_links, desc='Crawling'):
            crawl_data(link, DATA_DIR)

    for link in links3:
        news_links = []

        for i in tqdm.tqdm(list(range(MIN_PAGE, MAX_PAGE + 1)), desc=link):
            sub_link = link + '-p' + str(i)  # Topic link with page number
            news_links += get_links(sub_link, 'h3')

        for link in tqdm.tqdm(news_links, desc='Crawling'):
            crawl_data(link, DATA_DIR)
except Exception as e:
    print(sys.exc_info()[2])


Crawling:   2%|▏         | 13/570 [00:23<15:27,  1.66s/it]

Exception AttributeError has happened when crawl data from URL: https://video.vnexpress.net/tin-tuc/du-lich/du-khach-do-xo-chup-hinh-voi-mai-anh-dao-o-da-lat-4563608.html


Crawling:   3%|▎         | 16/570 [00:29<17:18,  1.87s/it]

Exception AttributeError has happened when crawl data from URL: https://video.vnexpress.net/tin-tuc/thoi-su/chua-tam-chuc-dong-nghit-nguoi-du-xuan-4563339.html


Crawling:   9%|▉         | 50/570 [01:28<14:55,  1.72s/it]

...

Crawling:  99%|█████████▉| 564/570 [17:22<00:10,  1.75s/it]

Exception AttributeError has happened when crawl data from URL: https://video.vnexpress.net/tin-tuc/the-thao/buoi-tap-cuoi-cung-cua-hlv-park-tren-san-my-dinh-4559436.html


Crawling: 100%|█████████▉| 569/570 [17:33<00:01,  1.90s/it]

Exception AttributeError has happened when crawl data from URL: https://video.vnexpress.net/tin-tuc/the-thao/5-nam-ong-park-giup-viet-nam-pha-dop-so-thai-lan-4559609.html


Crawling:  22%|██▏       | 131/589 [03:35<12:54,  1.69s/it]

### Crawl bằng thư viện *newspaper*

In [ ]:
import yaml
from yaml.loader import SafeLoader


with open('newspaper_link.yaml', 'r') as f:
    data = list(yaml.load_all(f, Loader=SafeLoader))
    vietnamese_link = data[0]['Vietnamese']

print(vietnamese_link)


In [ ]:
import newspaper


url = 'https://vnexpress.net/kich-ban-nao-cho-duc-o-vong-cuoi-world-cup-2022-4541291.html'

article = newspaper.Article(url)
article.download()
article.parse()

print(article.publish_date)


In [ ]:
import newspaper
import yaml
import os


CURRENT_DIR = Path(os.getcwd())
DATA_DIR = CURRENT_DIR / 'data'
MAX_ARTICLE_COUNT = 100

for link in vietnamese_link:
  news_name = link.split('//')[1].split('.')[0]

  news_paper = newspaper.build(link)
  if not Path(DATA_DIR / news_name).is_dir():
    os.mkdir(str(DATA_DIR / news_name))

  article_count = 0

  for article in news_paper.articles:
    article_count += 1
    if article_count > MAX_ARTICLE_COUNT:
      break

    # print(article.text)
    
    url = article.url
    article = newspaper.Article(url)

    article.download()
    article.parse()

    print(article.url)
